### Import Automotive Data and Cleanse

Machine Learning for Engineers: [Automotive Monitoring](https://www.apmonitor.com/pds/index.php/Main/AutomotiveMonitoring)
- Description: Machine learning project with automotive data. Data includes travel distance, time, fuel rate, air flow, oxygen ratio, and other parameters available from an OBD2 interface.
- [Course Overview](https://apmonitor.com/pds)
- [Course Schedule](https://apmonitor.com/pds/index.php/Main/CourseSchedule)

<img width=400px align=left src='https://apmonitor.com/pds/uploads/Main/automotive_monitoring.png'>

### Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Import Data and View Columns

Import `auto_warmup.csv`, set time index, and print data columns

In [ ]:
url = 'http://apmonitor.com/pds/uploads/Main/auto_warmup.txt'
data = pd.read_csv(url)

# set time index
data['time'] = pd.to_datetime(data['time'])
data = data.set_index('time')

# print data columns
for x in data.columns:
    print(x)

### Select Columns of Interest

In [ ]:
# select columns of interest
data1 = data[['Engine coolant temperature (℉)']].copy()
data1.dropna(inplace=True)
data2 = data[['Catalyst temperature Bank 1 Sensor 1 (℉)']].copy()
data2.dropna(inplace=True)
data3 = data[['Vehicle speed (mph)']].copy()

# join data
data = data1.join(data2,how='outer',sort=True)
data = data.join(data3,how='outer',sort=True)
data.columns = ['coolant (degF)','catalyst (degF)','speed (mph)']
data.head()
    
# fill in NaNs - forward fill
data.fillna(method='ffill',inplace=True)
# fill in NaNs - backward fill
data.fillna(method='bfill',inplace=True)
dr = len(data)
data.describe()

### Plot Data

In [ ]:
data.plot(subplots=True,figsize=(10,6))

### Remove Gross Outliers

In [ ]:
# data cleansing
data = data[data['coolant (degF)']>40]
data = data[data['speed (mph)']<20]

data['d1'] = data['coolant (degF)'].diff().abs()
data['d2'] = data['catalyst (degF)'].diff().abs()
data.plot(subplots=True,figsize=(10,6))

### Remove Other Outliers and Bad Speed Data

In [ ]:
# 3 cycles because cat temperature stays high for a few samples
for i in range(3):
    data['d1'] = data['coolant (degF)'].diff().abs()
    data['d2'] = data['catalyst (degF)'].diff().abs()
    data = data[data['d1']<5]
    data = data[data['d2']<10]

# zero speed between 6:30 to 6:50
data.iloc[data.index.indexer_between_time('06:30','06:50'), 2] = 0

data.plot(subplots=True,figsize=(10,6))
print('Rows removed: ',dr-len(data), ' of ', dr)

### Insert Feature (Engine On/Off) and Reduce Data

In [ ]:
# delete d1 and d2
del data['d1']
del data['d2']

# add engine state (on/off)
data['engine'] = 0
data.iloc[data.index.indexer_between_time('06:24:46','06:55:01'), 3] = 1

### Display Cleansed Data

In [ ]:
data.plot(subplots=True,figsize=(8,5))
plt.savefig('auto_warmup.png',dpi=300)
data.to_csv('auto_clean.csv')

### Determine Time to Catalyst Light-off

In [ ]:
select = ['engine','catalyst (degF)']
s = data[select].between_time('06:24:40','06:26:00').copy()
s['light-off'] = (s['catalyst (degF)']>500).astype(int)
s.plot(subplots=True)

In [ ]:
# find engine start time
for i in range(1,len(s)):
    if s['engine'].iloc[i]==1:
        engine_start = s.index[i]
        break

# find light-off time
for i in range(1,len(s)):
    if s['light-off'].iloc[i]==1:
        lightoff_start = s.index[i]
        break

# calculate seconds to catalyst light-off
dt = lightoff_start - engine_start
print(dt.total_seconds(), 'sec to catalyst light-off')        

### Create ARX Model

In [ ]:
# reduce data to about every 1-2 sec
data = data[::10].copy()

data['tmin'] = data.index
t0 = data['tmin'].iloc[0]
dt = (data['tmin'].copy()-t0)
dt2 = []
for i in range(len(data)):
    dt2.append(dt[i].total_seconds()/60.0)
data['tmin'] = dt2

In [ ]:
from gekko import GEKKO

t = data['tmin'].values
u = data[['engine','speed (mph)']]
y = data[['catalyst (degF)','coolant (degF)']]

# generate time-series model
m = GEKKO(remote=False)

# system identification
na = 2 # output coefficients
nb = 40 # input coefficients
yp,p,K = m.sysid(t,u,y,na,nb,pred='meas')

plt.figure(figsize=(10,6))
plt.subplot(2,1,1)
plt.plot(t,u)
plt.legend(['Engine','Speed (mph)'],loc=2)
plt.ylabel('Inputs')
plt.subplot(2,1,2)
plt.plot(t,y)
plt.plot(t,yp,'--')
plt.legend(); plt.ylabel('Outputs')
plt.legend(['Catalyst (degF) Meas','Coolant (degF) Meas',\
            'Catalyst (degF) Pred','Coolant (degF) Pred'],loc=4)

plt.xlabel('Time (min)'); plt.savefig('sysid.png')
plt.show()